In [1]:
#
%load_ext autoreload
%autoreload 2

import sys; sys.path.insert(0, '../..') # add parent folder path where lib folder is

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.metrics import r2_score, mean_squared_error
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score


from utils import helper, config, rayer, kaggle_dataset_helper
from ml.models.base.v2.brisk_xgboost import BriskXGBoost
from ml.models.base.v2.brisk_bagging import BriskBagging
from ml.models.base.v2.slug_xgboost import SlugXGBoost
from ml.models.base.v2.slug_knn import SlugKNN
from ml.models.base.v2.slug_rf import SlugRF


from ml.models import common


import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150) # print: change width of display
pd.options.display.max_colwidth = 100 # print: max width column
import warnings
warnings.filterwarnings('ignore')


In [6]:
df = helper.get_tokamat_dataset()

/home/wasif/python-asd/xai/notebooks/fusion/../../utils


In [7]:
df.head()

,TOK_ID,DIVNAME,LCUPDATE,DATE,SHOT,TIME,TIME_ID,AUXHEAT,PHASE,ISEQ,MEFF,PGASA,PGASZ,BGASA,BGASZ,BGASA2,BGASZ2,PELLET,XGASA,XGASZ,CONFIG,RGEO,AMIN,KAPPA,KAPPAA,KAREA,DELTA,INDENT,AREA,VOL,SURFFORM,SEPLIM,WALMAT,DIVMAT,LIMMAT,EVAP,BT,IP,VSURF,Q95,NEL,NELFORM,DNELDT,POHM,ENBI,PINJ,BSOURCE,PINJ2,BSOURCE2,PNBI,PFLOSS,PECRHC,PECRH,ICFREQ,ICSCHEME,ICANTEN,PICRHC,PICRH,PALPHA,DWHC,WFFORM,ICFORM,WTOT,WTH,PL,PLTH,TAUTOT,TAUTH,TAUC92,TAUC93,TOK
0,1,DV-IPRE,19940125,19820622,5980,0.959961,960,NONE,OHM,NONE,2.0,2.0,1,0,0,0,0,NONE,0.0,0,DN,1.667969,0.384033,1.041992,1.041992,1.041992,0.0,0.0,0.482666,5.058594,25.828125,0.208252,SS,TI2,NONE,NONE,2.203125,293900,1.073242,3.740234,-9223372036854775808,0.0,5.493e+18,315400,0.0,0,0.0,0,0,0,0,0,0,0,NONE,NONE,0,0,0,0.0,0,0.0,27950,27950,315400,315400,0.088623,0.088623,1.000000,1.000000,ASDEX
1,1,DV-IPRE,19940125,19820622,5980,1.009766,1010,NONE,OHM,NONE,2.0,2.0,1,0,0,0,0,NONE,0.0,0,DN,1.667969,0.383789,1.041992,1.041992,1.041992,0.0,0.0,0.482666,5.058594,25.812500,0.208252,SS,TI2,NONE,NONE,2.203125,294300,1.106445,3.734375,-9223372036854775808,0.0,8.423e+18,325500,0.0,0,0.0,0,0,0,0,0,0,0,NONE,NONE,0,0,0,0.0,0,0.0,28610,28610,325500,325500,0.087891,0.087891,1.000000,1.000000,ASDEX
2,1,DV-IPRE,19940125,19820622,5980,1.059570,1060,NONE,OHM,NONE,2.0,2.0,1,0,0,0,0,NONE,0.0,0,DN,1.667969,0.384033,1.041992,1.041992,1.041992,0.0,0.0,0.482666,5.058594,25.828125,0.208496,SS,TI2,NONE,NONE,2.203125,294400,1.015625,3.734375,-9223372036854775808,0.0,-1.904e+19,299200,0.0,0,0.0,0,0,0,0,0,0,0,NONE,NONE,0,0,0,0.0,0,0.0,28860,28860,299200,299200,0.096436,0.096436,1.000000,1.000000,ASDEX
3,1,DV-IPRE,19940125,19820622,5980,1.185547,1186,NB,H,NONE,1.5,2.0,1,1,1,0,0,NONE,0.0,0,DN,1.698242,0.406494,0.984375,0.984375,0.983887,0.0,0.0,0.510742,5.449219,27.031250,0.155640,SS,TI2,NONE,NONE,2.205078,298000,-0.069824,4.343750,-9223372036854775808,0.0,1.307e+20,0,41984.0,2620000,404020.0,0,0,2596000,570700,0,0,0,NONE,NONE,0,0,0,1.0,23430,0.0,103500,80110,1323000,752300,0.078247,0.106506,0.833496,0.833496,ASDEX
4,1,DV-IPRE,19940125,19820622,5980,1.205078,1205,NB,HGELM,NONE,1.5,2.0,1,1,1,0,0,NONE,0.0,0,DN,1.694336,0.410156,0.975098,0.975098,0.974609,0.0,0.0,0.515137,5.480469,27.093750,0.156372,SS,TI2,NONE,NONE,2.205078,295900,0.232056,4.562500,-9223372036854775808,0.0,-1.959e+19,68660,41984.0,2620000,404020.0,0,0,2599000,585700,0,0,0,NONE,NONE,0,0,0,1.0,24260,0.0,119600,95370,2456000,1870000,0.048706,0.050995,0.666504,0.666504,ASDEX
